# Exercise 15: Power analyses

This  assignment is designed to give you practice with Monte Carlo methods to conduct power analyses via simulation. You won't need to load in any data for this homework. We will, however, be using parts of the homework from last week.

1. Simulating data 1/1
2. run_analysis function 2/2
3. repeat_analysis() function 3/3
4. Testing different sample sizes 2/2
5. Reflection 2/2

---
## 1. Simulating data (1 points)


Pull your `simulate_data()` function from your last homework and add it below.

As a reminder, this function simulates the relationship between age, word reading experience, and reading comprehension skill.

`c` is reading comprehension, and `x` is word reading experience.

In [ ]:
sample_size = 100 # How many children in data set?
age_lo = 80     # minimum age, in months
age_hi = 200    # maximum age, in months
beta_xa = 0.5   # amount by which experience changes for increase of one month in age
beta_x0 = -5    # amount of experience when age = 0 (not interpretable, since minimum age for this data is 80 months)
sd_x = 50       # standard dev of gaussian noise term, epsilon_x
beta_ca = 0.8   # amount that comprehension score improves for every increase of one unit in age
beta_cx = 3     # amount that comprehension score improves for every increase of one unit in reading experience
beta_c0 = 10    # comprehension score when reading experience is 0.
sd_c = 85      # standard dev of gaussian noise term, epsilon_c

simulate_data <- function(sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c) {
      # WRITE YOUR CODE HERE

      #generate ages
      gen_ages <- runif(sample_size,age_lo,age_hi)
      #generate noise
      gen_eps_x <- rnorm(sample_size,0,sd_x)
      gen_eps_c <- rnorm(sample_size,0,sd_c)
      #linear combination of generated values to get x
      gen_x <- beta_xa*gen_ages + beta_x0 + gen_eps_x
      #then to get c
      gen_c <- beta_ca*gen_ages + beta_cx*gen_x + beta_c0 + gen_eps_c


      return(data.frame(age=gen_ages,x=gen_x,c=gen_c)) # it's actually bad form to have a variable named "c" in R, my bad...
}

set.seed(2)
dat <- simulate_data(sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)
head(dat)

,age,x,c
,<dbl>,<dbl>,<dbl>
1,102.1859,4.178578,187.23514
2,164.2849,180.457510,668.39947
3,148.7992,41.287227,314.28731
4,100.1662,108.868891,344.96408
5,193.2607,39.251729,390.93373
6,193.2170,-6.685415,30.35296


---
## 2. `run_analysis()` function (2 pts)

Last week, we looked at whether word reading experience(`x`) mediated the relation between `age` and reading comprehension (`c`).

Now we're going to use our `simulate_data()` function to conduct a power analysis. The goal is to determine how many participants we would need in order to detect both the mediated and the direct effects in this data.

*Note: We're going to pretend for the sake of simplicity that we don't have any control over the ages of the children we get (so ages are generated using `runif(sample_size, age_lo, age_hi)`, although of course this would be an unusual situation in reality.*

First, write a function, `run_analysis()`, that takes in simulated data, runs **your mediation from last week**, and returns a vector containing the ACME and ADE estimates and p-values (these are the `d0`, `d0.p`, `z0`, and `z0.p` features of the mediated model object, e.g., `fitMed$d0.p`). Print this function's output for the data we simulated previously.

In [ ]:
# WRITE YOUR CODE HERE
#install.packages("mediation")
library(mediation)


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘rbibutils’, ‘checkmate’, ‘Rdpack’, ‘zoo’, ‘gridExtra’, ‘htmlTable’, ‘viridis’, ‘Formula’, ‘minqa’, ‘nloptr’, ‘reformulas’, ‘RcppEigen’, ‘mvtnorm’, ‘sandwich’, ‘lpSolve’, ‘Hmisc’, ‘lme4’


Loading required package: MASS

Loading required package: Matrix

Loading required package: mvtnorm

Loading required package: sandwich

mediation: Causal Mediation Analysis
Version: 4.5.0




In [ ]:
run_analysis <- function(in_dat) {
  fitM <- lm(x ~ age,     data=in_dat) #Step 1: IV on M, Age predicting x
  fitY <- lm(c ~ x + age, data=in_dat) #Step 2: IV and M on DV, Number of passengers and weight predicting highway
  #summary(fitM)
  #summary(fitY)
  fitMed <- mediate(fitM, fitY, treat="age", mediator="x")
  #summary(fitMed)
  return(c(fitMed$d0, fitMed$d0.p, fitMed$z0, fitMed$z0.p))
}

run_analysis(dat)

[1] 1.386049 0.002000 1.191023 0.000000

---
## 3. `repeat_analysis()` function (3 pts)

Next fill in the function `repeat_analysis()` below so that it simulates and analyzes data `num_simulations` times. Store the outputs from each simulation in the `simouts` matrix. Calculate and return the coverage across all the simulations run for both ACME and ADE.

In [ ]:
repeat_analysis <- function(num_simulations, alpha, sample_size, age_lo, age_hi,
        beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c) {
    # Initialize simouts matrix for storing each output from run_analysis()
    simouts <- matrix(rep(NA, num_simulations*4), nrow=num_simulations, ncol=4)

    # Start simulating
    for (i in 1:num_simulations) {
      # WRITE YOUR CODE HERE
      hold_dat <- simulate_data(sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)
      simouts[i,]<-run_analysis(in_dat = hold_dat)
    }

    # Calculate coverage for both ACME and ADE estimates using p-values in simouts
    # FILL THIS IN
    ACME_cov = mean(simouts[,2] <= alpha)
    # FILL THIS IN
    ADE_cov = mean(simouts[,4] <= alpha)

    return(list(ACME_cov = ACME_cov, ADE_cov = ADE_cov))
}

Now run the `repeat_analysis()` function using the same parameter settings as above, for 10 simulations, with an alpha criterion of 0.01.

In [ ]:
# WRITE YOUR CODE HERE
num_simulations = 10
alpha = 0.01

repeat_analysis(num_simulations, alpha, sample_size, age_lo, age_hi,
        beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)

$ACME_cov
[1] 0.9

$ADE_cov
[1] 0.8

There is a 90% probability that we correctly reject the null about ACME (no mediation effect) at p < 0.01 if we measure the age, x, and c and 100 children per group.

And there is a 80% probability that we correctly reject the null about ADE (no direct effect) at p < 0.01 if we measure the age, x, and c and 100 children per group.

---
## 4. Testing different sample sizes (2 pts)

Finally, do the same thing (10 simulations, alpha criterion of 0.01) but for 5 different sample sizes: 50, 75, 100, 125, 150. You can do this using `map` (as in the tutorial), or a simple `for` loop, or by calculating each individually. Up to you! This should take around 3 minutes to run.

In [ ]:
# WRITE YOUR CODE HERE
list_sample_sizes <- c(50, 75, 100, 125, 150)
repeat_for_sample_sizes <- matrix(rep(NA, length(list_sample_sizes)*2), nrow=length(list_sample_sizes), ncol=2)

for(i in 1:length(list_sample_sizes)){
  cat("For analysis using sample size:", vary_samp_size)
  hold<-repeat_analysis(num_simulations, alpha, sample_size=list_sample_sizes[i], age_lo, age_hi,
        beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)
  print(hold)
  repeat_for_sample_sizes[i,1]<-hold$ACME_cov
  repeat_for_sample_sizes[i,2]<-hold$ADE_cov
}

For analysis using sample size: 50$ACME_cov
[1] 0.7

$ADE_cov
[1] 0.2

For analysis using sample size: 50$ACME_cov
[1] 0.6

$ADE_cov
[1] 0.5

For analysis using sample size: 50$ACME_cov
[1] 0.8

$ADE_cov
[1] 0.6

For analysis using sample size: 50$ACME_cov
[1] 0.9

$ADE_cov
[1] 0.7

For analysis using sample size: 50$ACME_cov
[1] 1

$ADE_cov
[1] 1



Print your results.

In [ ]:
# WRITE YOUR CODE HERE
repeat_for_sample_sizes

0.7,0.2
0.6,0.5
0.8,0.6
0.9,0.7
1.0,1.0


## 5. Reflection (2 pts)

If this were a real power analysis, we'd want to run more simulations per sample size (to get a more precise estimate of power) and we may also want to test out some other values of the parameters we used to simulate our data. However, what would you conclude just based on the results above?

> *Write your response here.*
>

I would conclude that with larger sample sizes, we get higher coverage (more power). In which case, we should probably collect a larger sample size if possible to get more power to detect the effect.
Also, it looks like our power is generally smaller for detecting the direct effect than the indirect effect (since the coverage is larger for ACME than ADE generally).

**Given** how we generated the data, why was the direct effect harder to detect than the mediated effect?
> *Write your response here.*

I guess that the direct effect is harder to detect than the mediated effect because the mediated effect seems to be a larger magnitude change (per unit change in age) since beta_cx = 3 and beta_xa = 0.5, which means that per unit change in age, I would guess that the indirect effect on c is about (on average) 1.5 units while the direct effect of age on c is related to beta_ca = 0.8 which is much smaller.

**DUE:** 5pm EST, April 3, 2024

**IMPORTANT** Did you collaborate with anyone on this assignment? If so, list their names here.
> *Someone's Name*